# Leaflet cluster map of talk locations

Assuming you are working in a Linux or Windows Subsystem for Linux environment, you may need to install some dependencies. Assuming a clean installation, the following will be needed:

```bash
sudo apt install jupyter
sudo apt install python3-pip
pip install python-frontmatter getorg --upgrade
```

After which you can run this from the `_talks/` directory, via:

```bash
 jupyter nbconvert --to notebook --execute talkmap.ipynb --output talkmap_out.ipynb
```
 
The `_talks/` directory contains `.md` files of all your talks. This scrapes the location YAML field from each `.md` file, geolocates it with `geopy/Nominatim`, and uses the `getorg` library to output data, HTML, and Javascript for a standalone cluster map.

In [2]:
# Start by installing the dependencies
!pip install python-frontmatter getorg --upgrade
import json
import frontmatter
import glob
import getorg
from geopy import Nominatim
from geopy.exc import GeocoderTimedOut

Iywidgets and ipyleaflet support disabled. You must be in a Jupyter notebook to use this feature.
Error raised:
No module named 'ipyleaflet'
Check that you have enabled ipyleaflet in Jupyter with:
    jupyter nbextension enable --py ipyleaflet


In [36]:
def add_with_link(talk, key):
    url_key = "url_" + key
    if url_key in talk:
        url = talk[url_key]
        return f'<a href="{url}" target="_blank" rel="noopener noreferrer">{talk[key]}</a>'
    else:
        return talk[key]

def center(string):
    return '<p style="text-align: center"> ' + string + ' </p>'

In [41]:
# read talk list from json file
with open('_talks/talk_list.json') as json_file:
    data = json.load(json_file)

# Set the default timeout, in seconds
TIMEOUT = 5

# Prepare to geolocate
geocoder = Nominatim(user_agent="aleingrosso.github.io")
location_dict = {}
location = ""
permalink = ""
title = ""

# Perform geolocation
for talk in data:
    
    # Press on if the location is not present
    if 'location' not in talk:
        continue

    # Prepare the description
    date = talk['date']
    title = talk['title'].strip()
    location = talk['location'].strip()
    venue = talk['venue'].strip()
    description = f"{date}<br>{title}"
    description += "<br>" + add_with_link(talk, 'venue')
    if "institution" in talk:
        description += "<br>" + add_with_link(talk, 'institution')
    if "video" in talk:
        url_video = talk["video"]
        description += f'<br><a href="{url_video}" target="_blank" rel="noopener noreferrer">Video</a>'
    description = center(description)

    # print(description)
    
    # Geocode the location and report the status
    try:
        location_dict[description] = geocoder.geocode(location, timeout=TIMEOUT)
        print(description, location_dict[description])
    except ValueError as ex:
        print(f"Error: geocode failed on input {location} with message {ex}")
    except GeocoderTimedOut as ex:
        print(f"Error: geocode timed out on input {location} with message {ex}")
    except Exception as ex:
        print(f"An unhandled exception occurred while processing input {location} with message {ex}")

# Save the map
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)

<p style="text-align: center"> Upcoming: 28/10/2025<br>Generalization performance of narrow one-hidden layer networks in the teacher-student setting<br><a href="https://ui.ungpd.com/Events/60bfc7b5-c9d0-4e3e-9aec-1a6a62e1e805" target="_blank" rel="noopener noreferrer">2025 CHAIR Structured Learning Workshop</a><br><a href="https://www.chalmers.se/en/" target="_blank" rel="noopener noreferrer">Chalmers University of Technology</a> </p> Göteborg, Göteborgs Stad, Västra Götalands län, 411 10, Sverige
<p style="text-align: center"> 17/07/2025<br>Statistical Mechanics of transfer learning in the proportional regime<br><a href="https://statphys29.org/" target="_blank" rel="noopener noreferrer">Statphys29</a> </p> Firenze, Toscana, Italia
<p style="text-align: center"> 10/07/2025<br>Statistical Mechanics of transfer learning in the proportional regime<br><a href="https://bidsa.unibocconi.eu/learning-and-optimization-high-dimensions-statphys29-satellite-workshop" target="_blank" rel="noopener 

'Written map to talkmap/'